# Motivation and Monte Carlo Method
One of the challenges that investors face is determining the likelihood of the price movement - i.e., what is the probability that the price will reach a certain level. One way to address this challenge is to simulate the price movement using [Monte Carlo method](https://en.wikipedia.org/wiki/Monte_Carlo_method).

Briefly speaking, Monte Carlo method assumes that a certain phenomenon (price movement) is deterministic and that we know its probability distribution and hence we can simulate it. These assumptions are naive. Therefore, an analyst should be careful with interpretation of the result.

Let us begin our analysis. First, we will decide on what data we will work on and download it. Second, we will design our model based on [Geometric Brownian Motion](https://en.wikipedia.org/wiki/Geometric_Brownian_motion) and set key model parameters. Finally, we will run our simulation and plot the result.

In [ ]:
import math
import random
import io
import requests

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from dateutil.relativedelta import relativedelta

We will model the stock price of Apple for the next year using 1000 simulations.

In determining model parameters (expected annual return and expected annual volatility) we will use data from the last 5 years. The timeframe that we base our assumptions on is the most critical part of the model. An analyst should take a timeframe that he or she determines to be the most similar to the circumstances in the forecast period or provide the value of expected annual return and expected annual volatility manually.

In [ ]:
ticker = 'aapl.us'  # Use ticker format from https://stooq.com
interval_prediction = 1  # Forecast in years
number_of_simulations = 1000  # Set desired number of simulations
look_back_x_years = 5

Next, we download the latest historic data on the prices of the stock, perform some data cleaning, and select the timeframe that interests us.

In [ ]:
target_url = 'https://stooq.com/q/d/l/?s={}&i=d'.format(ticker)
ticker_data = requests.get(target_url)
ticker_data = ticker_data.text
buffer = io.StringIO(ticker_data)
ticker_dataframe = pd.read_csv(buffer).drop(['Open', 'High', 'Low', 'Volume'], axis=1)
ticker_dataframe['Date'] = pd.to_datetime(ticker_dataframe['Date'], format='%Y-%m-%d', errors='ignore')

In [ ]:
pointer_date = ticker_dataframe['Date'].iloc[-1] + relativedelta(years = -look_back_x_years)

ticker_dataframe = ticker_dataframe[ticker_dataframe['Date'] >= pointer_date]

Let's see what our data looks like.

In [ ]:
ticker_dataframe.head()

In [ ]:
sns.lineplot(data=ticker_dataframe, x='Date', y='Close', color = (0.17, 0.74, 0.45))

# Making the Model
## Monte Carlo Method Function
Our Monte Carlo function is very simple - it runs given function a set number of times, records each iteration, and returns a list of results.

## Geometric Brownian Motion Function
On the other hand, the Geometric Brownian Motion function is a bit more complex. The equations we are using are derived from a research paper by Reddy and al. 2016.

The final price is obtained from the following equation 

$$S_{t + \Delta t}=S_{t} \exp[(\mu - \frac{\hat{\sigma}^{2}}{2})\Delta t + \hat{\sigma}\epsilon\sqrt{\Delta t}]$$

Where:
$$t \text{ - starting point in time}$$
$$\Delta t \text{ - forecast period}$$
$$S_{t} \text{ - price in t moment in time}$$
$$S_{t + \Delta t} \text{ - price at the end of the forecast period}$$
$$\mu \text{ - expected annual rate of return; in this example I use CAGR}$$
$$\hat{\sigma} \text{ - annualized expected annual volatility}$$
$$\epsilon \text{ - random volatility obtained as a random number drown from N(0, 1)}$$

To get the annualized expected annual volatility we use the following equation

$$\hat{\sigma} = \frac{s}{\sqrt{\tau}}$$

$$\tau = \frac{\Delta t}{N}$$

Where:
$$s \text{ - standard deviation of daily rate of return}$$
$$\tau \text{ - interval measured in years}$$
$$N \text{ - number of trading days in interval}$$

In [ ]:
def simple_monte_carlo(function, args, number_of_simulations):
    results = list()
    for x in range(number_of_simulations):
        results.append(function(*args))
    return results


def geometric_brownian_motion_stock_price(lastest_stock_price, expected_annual_return, expected_annual_volatility, interval_prediction):
    nrandom = random.normalvariate(0, 1)
    return lastest_stock_price * math.exp((expected_annual_return - 0.5 * expected_annual_volatility ** 2) * interval_prediction + expected_annual_volatility * nrandom * math.sqrt(interval_prediction))


def get_expected_annual_volatility(stock_prices):
    stock_daily_return = np.diff(stock_prices) / stock_prices[:len(stock_prices) - 1]
    stock_daily_return_sd = np.std(stock_daily_return)
    return stock_daily_return_sd / math.sqrt(look_back_x_years / len(stock_prices))

Next, we calculate model's key parameters.

In [ ]:
lastest_stock_price = ticker_dataframe['Close'].iloc[-1]
expected_annual_return = (ticker_dataframe['Close'].iloc[-1] / ticker_dataframe['Close'].iloc[0]) ** (1 / look_back_x_years) - 1  # CAGR
expected_annual_volatility = get_expected_annual_volatility(ticker_dataframe['Close'])

args = (lastest_stock_price, expected_annual_return, expected_annual_volatility, interval_prediction)

# Running the Simulation
There is nothing left to do but to run the simulation and plot the result.

In [ ]:
monte_carlo_simulation = simple_monte_carlo(geometric_brownian_motion_stock_price, args, number_of_simulations)

In [ ]:
sns.displot(monte_carlo_simulation, color = (0.17, 0.74, 0.45))

# Wrapping Up
We downloaded the data, prepared the model, and ran the simulation. From this point, we could start looking for percentiles that are of interest to us.

In [ ]:
percentiles = [5, 25, 50, 75, 95]
for percentile in percentiles:
    x = np.percentile(monte_carlo_simulation, percentile)
    print(f'{percentile} percentile: {x}')

# References
Reddy, Krishna and Clinton, Vaughan, Simulating Stock Prices Using Geometric Brownian Motion: Evidence from Australian Companies, Australasian Accounting, Business and Finance Journal, 10(3), 2016, 23-47. [doi:10.14453/aabfj.v10i3.3](http://dx.doi.org/10.14453/aabfj.v10i3.3)